In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB:
    %pip install -q -U tensorflow-addons
    %pip install -q -U transformers

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [2]:
# csv to df
path = os.path.abspath('politifact_balanced_data.csv')
df = pd.read_csv(path, index_col=0)

# specify rows of importance
df = df[["veracity", "statement"]]

# See sample of data
df.head(5)

,veracity,statement
3,0,"""Tim Kaine doesn’t want a border at all. He wa..."
5,0,"""The deficit ... is coming down, and it’s comi..."
20,1,"""Migrant mother and ‘crying girl’ on Time cove..."
23,0,"""Fact: Over 90,000 kids were detained under Ob..."
24,0,"""$1 billion—that’s how much Bruce Rauner has w..."


In [4]:
# manually split the dataset into train and test
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2165 entries, 3 to 11183
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   veracity   2165 non-null   int64 
 1   statement  2165 non-null   object
dtypes: int64(1), object(1)
memory usage: 50.7+ KB


In [31]:
train=df.sample(frac=0.65,random_state=200) #random state is a seed value
df_validation = df.drop(train.index)
validation=df_validation.sample(frac=0.6,random_state=200)
test = df_validation.drop(validation.index)

In [32]:
train.info()
validation.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1407 entries, 698 to 5808
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   veracity   1407 non-null   int64 
 1   statement  1407 non-null   object
dtypes: int64(1), object(1)
memory usage: 33.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 455 entries, 3246 to 4199
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   veracity   455 non-null    int64 
 1   statement  455 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 20 to 11121
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   veracity   303 non-null    int64 
 1   statement  303 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.1+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433 entries, 70 to 11183
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   veracity   433 non-null    int64 
 1   statement  433 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [8]:
X_train = train["statement"].copy()
y_train = train["veracity"].copy()
X_test = test["statement"].copy()
y_test = test["veracity"].copy()

# Tokenizing the dataset using keras

In [46]:
#include and cite
# https://www.kdnuggets.com/2020/03/tensorflow-keras-tokenization-text-data-prep.html

## Split Words with text_to_word_sequence

In [54]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words = 10000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

# Tokenize our training data
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(X_train)

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

# Output the results of our work
print("Word index:\n", word_index)
print("\nTraining sequences:\n", train_sequences)
print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(train_sequences))
print("Padded Training sequences data type:", type(train_padded))

Word index:
 {'<UNK>': 1, 'the': 2, 'in': 3, 'of': 4, 'to': 5, 'a': 6, 'and': 7, 'is': 8, 'that': 9, 'for': 10, 'have': 11, 'are': 12, 'than': 13, 'has': 14, 'percent': 15, 'on': 16, 'more': 17, 'we': 18, 'by': 19, 'state': 20, 'it': 21, 'years': 22, 'as': 23, 'was': 24, '000': 25, 'our': 26, 'not': 27, 'year': 28, 'from': 29, 'with': 30, 'this': 31, 'at': 32, 'people': 33, 'been': 34, 'their': 35, 'you': 36, 'i': 37, 'or': 38, 'over': 39, 'there': 40, 'states': 41, 'one': 42, 'they': 43, 'no': 44, 'every': 45, 'new': 46, 'country': 47, 'all': 48, 'million': 49, 'only': 50, 'an': 51, 'now': 52, 'since': 53, 'be': 54, '1': 55, 'tax': 56, 'u': 57, 'last': 58, 'were': 59, 's': 60, 'president': 61, 'out': 62, 'when': 63, 'who': 64, 'if': 65, 'will': 66, 'obama': 67, 'any': 68, 'health': 69, 'texas': 70, 'billion': 71, 'most': 72, 'time': 73, 'up': 74, 'about': 75, 'his': 76, 'money': 77, 'he': 78, 'had': 79, 'united': 80, 'jobs': 81, 'would': 82, 'rate': 83, 'taxes': 84, 'budget': 85, 'car

In [55]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

print("Testing sequences:\n", test_sequences)
print("\nPadded testing sequences:\n", test_padded)
print("\nPadded testing shape:",test_padded.shape)

Testing sequences:
 [[1518, 31, 28, 1394, 1, 416, 17, 13, 290, 49, 33, 3, 70, 7, 1259, 194, 679], [37, 11, 850, 1, 169, 68, 4, 257, 189, 1, 38, 1, 38, 2245, 232, 9], [173, 955, 4392, 303, 1, 145, 5, 938, 1], [719, 8, 1, 3064, 2909, 2, 1, 5, 2, 473, 439, 30, 1, 1, 1, 1, 1, 1940, 2237, 1, 190, 286, 7, 1, 246, 10, 48, 383, 18, 1, 1], [3645, 3507, 1, 2012, 3299, 19, 1, 127, 685, 1, 5, 1091], [2, 44, 55, 210, 1362, 3, 110, 45, 544, 10, 316, 22, 14, 34, 4500], [3621, 2779, 4359, 4, 158, 1, 217, 2682, 3, 3103, 1301, 1], [598, 15, 4, 33, 340, 3, 2, 1, 4404, 74, 870, 505, 1, 13, 35, 427, 115, 181, 340, 3, 2, 1456, 2, 234, 1, 2209, 11, 50, 6, 119, 119, 3855, 4, 870, 505, 1, 13, 35, 427, 2037, 217, 2, 1222, 1, 992, 3, 26, 156], [1, 92, 1, 142, 1, 651], [2, 494, 91, 256, 4887, 4888, 14, 34, 741, 1, 3, 446, 4, 1, 1, 2, 58, 536, 22], [53, 375, 206, 11, 774, 1, 308, 3, 45, 295, 298], [637, 445, 5, 1276, 2183, 651, 33, 64, 185, 1339, 398], [65, 36, 935, 2, 129, 665, 5162, 351, 21, 66, 54, 2, 244, 987,

for x, y in zip(X_test, test_padded):
  print('{} -> {}'.format(x, y))

print("\nWord index (for reference):", word_index)

# ==================

In [79]:
X_train.iloc[698]

'"When Congress voted to condemn Iran for sentencing a Christian pastor to death, (U.S. Rep. Lois) Capps was the only member who voted no."'

In [60]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 500)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [66]:
X_batch, y_batch = preprocess(X_train, y_train)

In [67]:
from collections import Counter

vocabulary = Counter()
for review in X_batch:
    vocabulary.update(list(review.numpy()))

In [68]:
vocabulary.most_common()[:3]

[(b'<pad>', 78515), (b'the', 1466), (b'in', 896)]

In [69]:
len(vocabulary)

5496

In [70]:
vocab_size = len(vocabulary)
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [71]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [80]:
table.lookup(tf.constant([b"When Congress voted to condemn Iran for sentencing a Christian pastor to death, (U.S. Rep. Lois) Capps was the only member who voted no".split()]))

<tf.Tensor: shape=(1, 24), dtype=int64, numpy=
array([[ 111,  133,  109,    4, 3627,  602,    9, 3628,    5, 3629, 3630,
           4, 5530, 6093, 6449, 5752, 3631,   22,    1,   52, 1179,   58,
         109,   65]], dtype=int64)>

In [84]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = train.apply(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

TypeError: preprocess() missing 1 required positional argument: 'y_batch'

# Transforming the statements into tensors

In [9]:
X_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 1732 entries, 698 to 4956
Series name: statement
Non-Null Count  Dtype 
--------------  ----- 
1732 non-null   object
dtypes: object(1)
memory usage: 27.1+ KB


In [12]:
train_size = len(X_train)
test_size = len(X_test)
print(train_size)
print(test_size)

1732
433


In [35]:
from tensorflow.keras.preprocessing.text import Tokenizer

train_text = train['statement'].to_numpy()
tok = Tokenizer(oov_token='<unk>')
tok.fit_on_texts(train_text)
tok.word_index['<pad>'] = 0
tok.index_word[0] = '<pad>'

train_seqs = tok.texts_to_sequences(train_text)
train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

train_labels = train['veracity'].to_numpy().flatten()

valid_text = validation['statement'].to_numpy()
valid_seqs = tok.texts_to_sequences(valid_text)
valid_seqs = tf.keras.preprocessing.sequence.pad_sequences(valid_seqs, padding='post')

valid_labels = validation['veracity'].to_numpy().flatten()



# CONVERT TO TF DATASETS

train_ds = tf.data.Dataset.from_tensor_slices((train_seqs,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_seqs,valid_labels))

BATCH_SIZE = 32
BUFFER_SIZE = 7
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_ds = valid_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [42]:
vocab_size=10000
embedding_dim = 70
model = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim),
    layers.GlobalAveragePooling1D(),
    layers.GRU(128, return_sequences=True),
    layers.GRU(128),
    layers.Dense(1, activation='sigmoid') # also tried activation='softmax'
])

model.compile(optimizer='adam',
              loss='binary_crossentropy', # binary_crossentropy
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=1,
    validation_data=valid_ds, validation_steps=1, steps_per_epoch=BUFFER_SIZE)

7/7 [==============================] - 0s 18ms/step - loss: 0.6918 - accuracy: 0.5357 - val_loss: 0.6918 - val_accuracy: 0.5625


In [46]:
embed_size = 128
num_oov_buckets = 1000
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_ds, epochs=5)

Epoch 1/5
44/44 [==============================] - 9s 86ms/step - loss: 0.6898 - accuracy: 0.5352
Epoch 2/5
44/44 [==============================] - 4s 86ms/step - loss: 0.4963 - accuracy: 0.7747
Epoch 3/5
44/44 [==============================] - 4s 87ms/step - loss: 0.1792 - accuracy: 0.9346
Epoch 4/5
44/44 [==============================] - 4s 86ms/step - loss: 0.0181 - accuracy: 0.9943
Epoch 5/5
44/44 [==============================] - 4s 88ms/step - loss: 0.0109 - accuracy: 0.9964


In [47]:
history.history

{'loss': [0.6898386478424072,
  0.49632391333580017,
  0.17916060984134674,
  0.018059005960822105,
  0.010916289873421192],
 'accuracy': [0.5351812243461609,
  0.7746979594230652,
  0.9346126317977905,
  0.9943141341209412,
  0.9964463114738464]}

In [50]:
test_text = test['statement'].to_numpy()
test_seqs = tok.texts_to_sequences(test_text)
test_seqs = tf.keras.preprocessing.sequence.pad_sequences(test_seqs, padding='post')

test_labels = test['veracity'].to_numpy().flatten()

results = model.evaluate(test_seqs, test_labels, batch_size=32)

10/10 [==============================] - 0s 23ms/step - loss: 2.1915 - accuracy: 0.4983


In [19]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [20]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [24]:
train_text = X_train
label = y_train
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

In [25]:
def binary_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return binary_vectorize_layer(text), label

In [26]:
def int_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return int_vectorize_layer(text), label

In [28]:
print(train)

      veracity                                          statement
698          1  "In my 30 year career including dozens of capi...
8080         0  "Your tax dollars are not being used to sue yo...
4138         1  "Women of childbearing age spend 68 percent mo...
510          0        "Sanctuary cities are extremely dangerous."
5003         0  "Local law enforcement . . . will have access ...
...        ...                                                ...
1579         1       "22 veterans take their own lives each day."
5827         1  "Even members of the NRA, when they were polle...
3622         0  "According to recent reports, a group of death...
4421         1  "Women in Oregon are paid 79 cents for every d...
4956         0  "If I have affordable coverage in my workplace...

[1732 rows x 2 columns]


In [27]:
text_batch, label_batch = next(iter(train))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

ValueError: too many values to unpack (expected 2)

# Testing the model without a validation set

In [4]:
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test = df.drop(train.index)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

train_text = train['statement'].to_numpy()
tok = Tokenizer(oov_token='<unk>')
tok.fit_on_texts(train_text)
tok.word_index['<pad>'] = 0
tok.index_word[0] = '<pad>'

train_seqs = tok.texts_to_sequences(train_text)
train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

train_labels = train['veracity'].to_numpy().flatten()


# CONVERT TO TF DATASETS

train_ds = tf.data.Dataset.from_tensor_slices((train_seqs,train_labels))

BATCH_SIZE = 32
BUFFER_SIZE = 7
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [7]:
embed_size = 128
num_oov_buckets = 1000
vocab_size=10000
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_ds, epochs=5)

Epoch 1/5
55/55 [==============================] - 24s 151ms/step - loss: 0.6836 - accuracy: 0.5508
Epoch 2/5
55/55 [==============================] - 8s 138ms/step - loss: 0.4578 - accuracy: 0.7846
Epoch 3/5
55/55 [==============================] - 8s 147ms/step - loss: 0.1819 - accuracy: 0.9342
Epoch 4/5
55/55 [==============================] - 7s 121ms/step - loss: 0.0526 - accuracy: 0.9815
Epoch 5/5
55/55 [==============================] - 8s 148ms/step - loss: 0.0310 - accuracy: 0.9902


In [8]:
history.history

{'loss': [0.6836201548576355,
  0.4578196406364441,
  0.18188466131687164,
  0.052647653967142105,
  0.031013986095786095],
 'accuracy': [0.550808310508728,
  0.7846420407295227,
  0.9341801404953003,
  0.9815242290496826,
  0.9901847839355469]}

In [9]:
test_text = test['statement'].to_numpy()
test_seqs = tok.texts_to_sequences(test_text)
test_seqs = tf.keras.preprocessing.sequence.pad_sequences(test_seqs, padding='post')

test_labels = test['veracity'].to_numpy().flatten()

results = model.evaluate(test_seqs, test_labels, batch_size=32)

14/14 [==============================] - 6s 74ms/step - loss: 2.0720 - accuracy: 0.5635


# Another approach

In [10]:
# put the csv dataset into 2 separate .txt files, one for each label

In [11]:
path = os.path.abspath('politifact_balanced_data.csv')
df = pd.read_csv(path, index_col=0)

# specify rows of importance
df = df[["veracity", "statement"]]

# See sample of data
df.head(5)

,veracity,statement
3,0,"""Tim Kaine doesn’t want a border at all. He wa..."
5,0,"""The deficit ... is coming down, and it’s comi..."
20,1,"""Migrant mother and ‘crying girl’ on Time cove..."
23,0,"""Fact: Over 90,000 kids were detained under Ob..."
24,0,"""$1 billion—that’s how much Bruce Rauner has w..."


In [53]:
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
import nltk
import numpy as np 
import random 
import pickle 
from collections import Counter 

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [49]:
#specific nltk downloads
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\catat\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\catat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\catat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
pos_df = df[df['veracity'] == 1]['statement']
neg_df = df[df['veracity'] == 0]['statement']
pos_df.info()
neg_df.info()

<class 'pandas.core.series.Series'>
Int64Index: 1070 entries, 20 to 11181
Series name: statement
Non-Null Count  Dtype 
--------------  ----- 
1070 non-null   object
dtypes: object(1)
memory usage: 16.7+ KB
<class 'pandas.core.series.Series'>
Int64Index: 1095 entries, 3 to 11183
Series name: statement
Non-Null Count  Dtype 
--------------  ----- 
1095 non-null   object
dtypes: object(1)
memory usage: 17.1+ KB


In [37]:

with open('positive.txt', 'w', encoding="utf-8") as f:
    dfAsString = pos_df.to_string(header=False, index=False)
    f.write(dfAsString)



In [38]:
with open('negative.txt', 'w', encoding="utf-8") as f1:
    dfAsString1 = neg_df.to_string(header=False, index=False)
    f1.write(dfAsString1)

In [45]:
lemmatizer = WordNetLemmatizer() 
max_lines = 10000000 
pos = 'positive.txt'
neg = 'negative.txt'
def create_lexicon(pos, neg): 
    lexicon = [] 
    for fi in [pos, neg]: 
        with open(fi, 'r') as f: 
            contents = f.readlines() 
            for l in contents[:max_lines]: 
                all_words = word_tokenize(l.lower()) 
                lexicon += list(all_words) 
 
    lexicon = [lemmatizer.lemmatize(i) for i in lexicon] 
    w_counts = Counter(lexicon) 
 
    l2 =[] 
    for w in w_counts: 
        if 1000 > w_counts[w] > 50: 
            l2.append(w) 
    return l2 
def sample_handling(sample,lexicon,classification): 
    featureset = [] 
    with open(sample,'r') as f: 
        contents = f.readlines() 
        for l in contents[:10000000]: 
            current_words = word_tokenize(l.lower()) 
            current_words = [lemmatizer.lemmatize(i) for i in 
            current_words] 
            features = np.zeros(len(lexicon)) 
            for word in current_words: 
                if word.lower() in lexicon: 
                    index_value = lexicon.index(word.lower()) 
                    features[index_value] += 1 
            features = list(features) 
            featureset.append([features,classification]) 
    return featureset 

In [50]:
lexicon = create_lexicon(pos,neg) 
features = [] 
features += sample_handling(pos, lexicon,[1,0]) 
features += sample_handling(neg, lexicon,[0,1]) 
random.shuffle(features) 
features = np.array(features) 

testing_size = int(0.1*len(features)) 

X_train = list(features[:,0][:-testing_size]) 
y_train = list(features[:,1][:-testing_size]) 
X_test = list(features[:,0][-testing_size:]) 
y_test = list(features[:,1][-testing_size:]) 

C:\Users\catat\AppData\Local\Temp\ipykernel_30472\1363967790.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.array(features)


In [51]:
n_epochs = 10 
batch_size = 128 
h1 = 500 
h2 = 500  
n_classes = 2 

In [54]:

x_input = tf.placeholder('float') 
y_input = tf.placeholder('float') 
 
hidden_1 = {'weight':tf.Variable(tf.random_normal([len(X_train[0]), h1])), 
                  'bias':tf.Variable(tf.random_normal([h1]))} 
 
hidden_2 = {'weight':tf.Variable(tf.random_normal([h1, h2])), 
                  'bias':tf.Variable(tf.random_normal([h2]))} 
  
output_layer = {'weight':tf.Variable(tf.random_normal([h2, n_classes])), 
                'bias':tf.Variable(tf.random_normal([n_classes])),} 

In [55]:
l1 = tf.add(tf.matmul(x_input, hidden_1['weight']), hidden_1['bias']) 
l1 = tf.nn.relu(l1) 

l2 = tf.add(tf.matmul(l1, hidden_2['weight']), hidden_2['bias']) 
l2 = tf.nn.relu(l2) 

output = tf.matmul(l2, output_layer['weight']) + output_layer['bias'] 

In [56]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y_input)) 
opt = tf.train.AdamOptimizer().minimize(loss) 


with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer()) 

    for epoch in range(n_epochs): 
        epoch_loss = 0 
        i = 0 
        while i < len(X_train): 
            start = i 
            end = i + batch_size 
            batch_x = np.array(X_train[start:end]) 
            batch_y = np.array(y_train[start:end]) 

            _, batch_loss = sess.run([opt, loss], feed_dict={x_input: batch_x, y_input: batch_y}) 
            epoch_loss += batch_loss
            i += batch_size 

        print('Epoch {}: loss {}'.format(epoch, epoch_loss))
        
        sess.eval()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0: loss 3149.9771270751953
Epoch 1: loss 1921.4804611206055
Epoch 2: loss 1388.9053802490234
Epoch 3: loss 1083.3934173583984
Epoch 4: loss 898.1759986877441
Epoch 5: loss 751.7393703460693
Epoch 6: loss 639.744921207428
Epoch 7: loss 549.2992619276047
Epoch 8: loss 481.9416726231575
Epoch 9: loss 443.7921423316002


In [57]:
a = tf.Variable(5, name="scalar")

TypeError: FlattenDictItems: 'dict' must be a dictionary or collection.Mapping type object, instead of 'list'.